## Imports

In [31]:
import datetime

import matplotlib.pyplot as plt
import holidays
import numpy as np

import lightgbm as lgb
import pandas as pd
import polars as pl
import pathlib
import mlflow
from sklearn.model_selection import TimeSeriesSplit

## Environment variables

In [32]:
DEBUG = False

In [33]:
import random
def set_seed(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    pl.set_random_seed(seed)

SEED = 42
set_seed(seed=SEED)

## Constants definition

### Paths

In [34]:
BASE_DATA_PATH: pathlib.Path = pathlib.Path().absolute().parent.parent / 'data' / 'predict-energy-behavior-of-prosumers'

CLIENTS_PATH: pathlib.Path = BASE_DATA_PATH / 'client.csv'
ELECTRICITY_PATH: pathlib.Path = BASE_DATA_PATH / 'electricity_prices.csv'
GAS_PATH: pathlib.Path = BASE_DATA_PATH / 'gas_prices.csv'
HISTORICAL_WEATHER_PATH: pathlib.Path = BASE_DATA_PATH / 'historical_weather.csv'
WEATHER_FORECAST_PATH: pathlib.Path = BASE_DATA_PATH / 'forecast_weather.csv'
TRAIN_PATH: pathlib.Path = BASE_DATA_PATH / 'train.csv'
WEATHER_STATION_COUNTY_PATH: pathlib.Path = BASE_DATA_PATH / 'weather_station_to_county_mapping.csv'
COUNTY_MAP_PATH: pathlib.Path = BASE_DATA_PATH / "county_id_to_name_map.json"

### Constants

In [35]:
DATE_FORMAT: str = '%Y-%m-%d'
DATETIME_FORMAT: str = '%Y-%m-%d %H:%M:%S'
TIMEZONE: str = "Europe/Tallinn"
ESTONIAN_HOLIDAYS = list(
    holidays.country_holidays("EE", years=range(2021, 2026)).keys()
)

## Helper functions

In [36]:
def load_clients(clients_df: pd.DataFrame = None) -> pl.LazyFrame:
    
    if not clients_df:
        clients: pl.LazyFrame = pl.scan_csv(CLIENTS_PATH)
    else:
        clients = pl.from_pandas(clients_df).lazy()
        
    return clients.with_columns(
        [
            pl.col("product_type").cast(pl.Int8),
            pl.col("county").cast(pl.Int8),
            pl.col("eic_count").cast(pl.Int16),
            pl.col("installed_capacity").cast(pl.Float32),
            pl.col("is_business").cast(pl.Int8),
            pl.col("date").str.to_date(DATE_FORMAT),
            pl.col("data_block_id").cast(pl.Int16)
        ]
    )

In [37]:
def load_electricity(electricity_df: pd.DataFrame = None) -> pl.LazyFrame:
    if not electricity_df:
        electricity: pl.LazyFrame = pl.scan_csv(ELECTRICITY_PATH)
    else:
        electricity = pl.from_pandas(electricity_df).lazy()

    electricity = electricity.drop(["origin_date"]).with_columns(
        [
            pl.col("forecast_date").str.to_datetime(DATETIME_FORMAT) + pl.duration(days=1),
            pl.col("euros_per_mwh").cast(pl.Float32),
            pl.col("data_block_id").cast(pl.Int16)
        ]
    ).rename({"forecast_date": "datetime", "euros_per_mwh": "electricity_euros_per_mwh"})
    return electricity

In [38]:
def load_gas(gas_df: pd.DataFrame = None) -> pl.LazyFrame:
    if not gas_df:
        gas: pl.LazyFrame = pl.scan_csv(GAS_PATH)
    else:
        gas: pl.LazyFrame = pl.from_pandas(gas_df).lazy()

    gas = gas.drop(["origin_date"]).with_columns(
        [
            pl.col("forecast_date").str.to_date(DATE_FORMAT),
            pl.col("lowest_price_per_mwh").cast(pl.Float32),
            pl.col("highest_price_per_mwh").cast(pl.Float32),
            ((pl.col("lowest_price_per_mwh") + pl.col("highest_price_per_mwh")) / 2).alias("gas_mean_price_per_mhw"),
            pl.col("data_block_id").cast(pl.Int16)
        ]
    ).rename({"forecast_date": "date", "lowest_price_per_mwh": "gas_lowest_price_per_mwh", "highest_price_per_mwh": "gas_highest_price_per_mwh"})
    return gas

In [39]:
def load_weather_county_map() -> pl.LazyFrame:
    weather_station_county_mapping: pl.LazyFrame = pl.scan_csv(WEATHER_STATION_COUNTY_PATH)
    weather_station_county_mapping = weather_station_county_mapping.with_columns([
        pl.col("longitude").cast(pl.Float32).round(decimals=2),
        pl.col("latitude").cast(pl.Float32).round(decimals=2),
        pl.col("county").cast(pl.Int8).fill_null(-1),
        pl.col("county_name").fill_null("Unknown")
    ])
    
    weather_station_county_mapping = weather_station_county_mapping.join(other=weather_station_county_mapping.group_by("county").agg([
        pl.col("longitude").min().alias("longitude_min"),
        pl.col("longitude").max().alias("longitude_max"),
        pl.col("latitude").min().alias("latitude_min"),
        pl.col("latitude").max().alias("latitude_max"),
    ]),
        on=["county"],
        how="inner"
    )

    return weather_station_county_mapping

In [40]:
def load_weather_forecast(weather_station_county_mapping: pl.LazyFrame, weather_forecast_df: pd.DataFrame = None) -> pl.LazyFrame:
    if not weather_forecast_df:
        weather_forecast: pl.LazyFrame = pl.scan_csv(WEATHER_FORECAST_PATH)
    else:
        weather_forecast: pl.LazyFrame = pl.from_pandas(weather_forecast_df).lazy()
    
    weather_forecast: pl.LazyFrame = weather_forecast.drop(["origin_datetime"]).rename({"forecast_datetime": "datetime"})
    # weather_forecast = weather_forecast.filter(pl.col("hours_ahead") >= 24)  # we don't need to forecast for today
    weather_forecast = weather_forecast.with_columns(
        [
            pl.col("datetime").str.to_datetime(DATETIME_FORMAT),
            pl.col("latitude").cast(pl.Float32).round(decimals=2),
            pl.col("longitude").cast(pl.Float32).round(decimals=2),
            pl.col("data_block_id").cast(pl.Int16),
        ]
    )
    
    weather_forecast = weather_forecast.join(
        other=weather_station_county_mapping,
        how="left",
        on=["latitude", "longitude"]
    ).drop(["latitude", "longitude"])
    
    weather_forecast_mean = weather_forecast.group_by("county", "datetime", "data_block_id").agg(
            pl.col("hours_ahead").mean(),
            pl.col("temperature").mean(),
            pl.col("dewpoint").mean(),
            pl.col("cloudcover_high").mean(),
            pl.col("cloudcover_low").mean(),
            pl.col("cloudcover_mid").mean(),
            pl.col("cloudcover_total").mean(),
            pl.col("10_metre_u_wind_component").mean(),
            pl.col("10_metre_v_wind_component").mean(),
            pl.col("direct_solar_radiation").mean(),
            pl.col("surface_solar_radiation_downwards").mean(),
            pl.col("snowfall").mean(),
            pl.col("total_precipitation").mean(),
            pl.col("latitude_min").first(),
            pl.col("latitude_max").first(),
            pl.col("longitude_min").first(),
            pl.col("longitude_max").first(),
            pl.col("county_name").first(),
    )
    
    return weather_forecast_mean

In [41]:
def load_historical_weather(weather_station_county_mapping: pl.LazyFrame, historical_weather_df: pd.DataFrame = None) -> pl.LazyFrame:
    if not historical_weather_df:
        historical_weather: pl.LazyFrame = pl.scan_csv(HISTORICAL_WEATHER_PATH)
    else:
        historical_weather: pl.LazyFrame = pl.from_pandas(historical_weather_df).lazy()
    
    historical_weather = historical_weather.with_columns(
        [
            pl.col("datetime").str.to_datetime(DATETIME_FORMAT),
            pl.col("latitude").cast(pl.Float32).round(decimals=2),
            pl.col("longitude").cast(pl.Float32).round(decimals=2),
            pl.col("data_block_id").cast(pl.Int16)
        ]
    )
    
    historical_weather = historical_weather.join(
        other=weather_station_county_mapping,
        how="left",
        on=["latitude", "longitude"]
    ).drop(["latitude", "longitude"])
    
    historical_weather_mean = historical_weather.group_by("county", "datetime", "data_block_id").agg(
        pl.col("temperature").mean(),
        pl.col("dewpoint").mean(),
        pl.col("rain").mean(),
        pl.col("snowfall").mean(),
        pl.col("surface_pressure").mean(),
        pl.col("cloudcover_total").mean(),
        pl.col("cloudcover_low").mean(),
        pl.col("cloudcover_mid").mean(),
        pl.col("cloudcover_high").mean(),
        pl.col("windspeed_10m").mean(),
        pl.col("winddirection_10m").mean(),
        pl.col("shortwave_radiation").mean(),
        pl.col("direct_solar_radiation").mean(),
        pl.col("diffuse_radiation").mean(),
        pl.col("latitude_min").first(),
        pl.col("latitude_max").first(),
        pl.col("longitude_min").first(),
        pl.col("longitude_max").first(),
        pl.col("county_name").first(),
        )
    
    # Test set has 1 day offset for hour<11 and 2 day offset for hour>11
    historical_weather_mean = historical_weather_mean.with_columns(
        pl.when(pl.col("datetime").dt.hour() < 11).then(pl.col("datetime") + pl.duration(days=1)).otherwise(pl.col("datetime") + pl.duration(days=2))
    )
    
    return historical_weather_mean

In [42]:
def load_train_data(train_df: pd.DataFrame = None) -> pl.LazyFrame:
    if not train_df:
        train: pl.LazyFrame = pl.scan_csv(TRAIN_PATH)   
    else:
        train: pl.LazyFrame = pl.from_pandas(train_df).lazy()
    
    train = train.drop(["prediction_unit_id", "row_id"]).with_columns(
            pl.col("datetime").str.to_datetime(DATETIME_FORMAT),
            pl.col("is_business").cast(pl.Int8),
            pl.col("product_type").cast(pl.Int8),
            pl.col("target").cast(pl.Float32),
            pl.col("is_consumption").cast(pl.Int8),
            pl.col("county").cast(pl.Int8),
            pl.col("data_block_id").cast(pl.Int16),
    )
    return train.with_columns(
            pl.col("datetime").dt.year().alias("year"),
            pl.col("datetime").cast(pl.Date).alias("date"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.ordinal_day().alias("day_of_year"),
            pl.col("datetime").dt.hour().alias("hour"),
    )

In [43]:
def create_lagged_weather_forecast_features(weather_forecast: pl.LazyFrame):
    lagged_weather_forecast = weather_forecast.sort("county", "datetime", "data_block_id").rolling(index_column="datetime", period=f"1d", by=["county", "data_block_id"]).agg(
        [
            pl.col("temperature").mean().alias("temperature_forecast_last_day"),
            pl.col("dewpoint").mean().alias("dewpoint_forecast_last_day"),
            pl.col("snowfall").mean().alias("snowfall_forecast_last_day"),
            pl.col("cloudcover_total").mean().alias("cloudcover_total_forecast_last_day"),
            pl.col("cloudcover_low").mean().alias("cloudcover_low_forecast_last_day"),
            pl.col("cloudcover_mid").mean().alias("cloudcover_mid_forecast_last_day"),
            pl.col("cloudcover_high").mean().alias("cloudcover_high_forecast_last_day"),
            pl.col("10_metre_u_wind_component").mean().alias("10_metre_u_wind_component_forecast_last_day"),
            pl.col("10_metre_v_wind_component").mean().alias("10_metre_v_wind_component_forecast_last_day"),
            pl.col("surface_solar_radiation_downwards").mean().alias("surface_solar_radiation_downwards_forecast_last_day"),
            pl.col("direct_solar_radiation").mean().alias("direct_solar_radiation_forecast_last_day"),
            pl.col("total_precipitation").mean().alias("total_precipitation_forecast_last_day"),
        ]
    )

    return lagged_weather_forecast

def create_lagged_historical_weather_week_features(historical_weather: pl.LazyFrame):
    lagged_historical_weather_week = historical_weather.sort("county", "datetime", "data_block_id").rolling(index_column="datetime", period=f"1w", by=["county"]).agg(
        [
            pl.col("temperature").mean().alias("temperature_last_7_days"),
            pl.col("dewpoint").mean().alias("dewpoint_last_7_days"),
            pl.col("rain").mean().alias("rain_last_7_days"),
            pl.col("snowfall").mean().alias("snowfall_last_7_days"),
            pl.col("cloudcover_total").mean().alias("cloudcover_total_last_7_days"),
            pl.col("cloudcover_low").mean().alias("cloudcover_low_last_7_days"),
            pl.col("cloudcover_mid").mean().alias("cloudcover_mid_last_7_days"),
            pl.col("cloudcover_high").mean().alias("cloudcover_high_last_7_days"),
            pl.col("windspeed_10m").mean().alias("windspeed_10m_last_7_days"),
            pl.col("winddirection_10m").mean().alias("winddirection_10m_last_7_days"),
            pl.col("shortwave_radiation").mean().alias("shortwave_radiation_last_7_days"),
            pl.col("direct_solar_radiation").mean().alias("direct_solar_radiation_last_7_days"),
            pl.col("diffuse_radiation").mean().alias("diffuse_radiation_last_7_days"),
        ]
    )

    return lagged_historical_weather_week


def create_lagged_historical_weather_day_features(historical_weather: pl.LazyFrame):
    lagged_historical_weather_day = historical_weather.with_columns(pl.col("datetime").dt.hour().alias("hour")).sort("county", "datetime").rolling(index_column="datetime", period=f"1d", by=["county"]).agg(
        [
            pl.col("temperature").mean().alias("temperature_last_24_hours"),
            pl.col("dewpoint").mean().alias("dewpoint_last_24_hours"),
            pl.col("rain").mean().alias("rain_last_24_hours"),
            pl.col("snowfall").mean().alias("snowfall_last_24_hours"),
            pl.col("cloudcover_total").mean().alias("cloudcover_total_last_24_hours"),
            pl.col("cloudcover_low").mean().alias("cloudcover_low_last_24_hours"),
            pl.col("cloudcover_mid").mean().alias("cloudcover_mid_last_24_hours"),
            pl.col("cloudcover_high").mean().alias("cloudcover_high_last_24_hours"),
            pl.col("windspeed_10m").mean().alias("windspeed_10m_last_24_hours"),
            pl.col("winddirection_10m").mean().alias("winddirection_10m_last_24_hours"),
            pl.col("shortwave_radiation").mean().alias("shortwave_radiation_last_24_hours"),
            pl.col("direct_solar_radiation").mean().alias("direct_solar_radiation_last_24_hours"),
            pl.col("diffuse_radiation").mean().alias("diffuse_radiation_last_24_hours"),
        ]
    )
    return lagged_historical_weather_day

In [44]:
def create_training_data(
        train_df: pd.DataFrame = None,
        clients_df: pd.DataFrame = None,
        gas_df: pd.DataFrame = None,
        electricity_df: pd.DataFrame = None,
        historical_weather_df: pd.DataFrame = None,
        weather_forecast_df: pd.DataFrame = None,
) -> pl.LazyFrame:
    client_ids_columns = ["county", "is_business", "product_type", "is_consumption"]
    data: pl.LazyFrame = (load_train_data(train_df)
                          .join(other=load_clients(clients_df), how="left", on=["county", "is_business", "product_type", "data_block_id"], suffix="_client")
                          .join(other=load_gas(gas_df), on="data_block_id", how="left", suffix="_gas")
                          .join(other=load_electricity(electricity_df), on=["datetime", "data_block_id"], how="left", suffix="_electricity")
                        )

    data = data.group_by(client_ids_columns).len().drop("len").sort(client_ids_columns).with_row_index(name="client_id").join(
        other=data,
        how="inner",
        on=["county", "is_business", "product_type", "is_consumption"]
    )

    weather_station_county_map = load_weather_county_map()
    historical_weather = load_historical_weather(weather_station_county_map, historical_weather_df)
    weather_forecast = load_weather_forecast(weather_station_county_map, weather_forecast_df)
    
    data = (data
              .join(other=historical_weather, how="left", on=["county", "datetime", "data_block_id"], suffix="_measured")
              .join(other=weather_forecast, how="left", on=["county", "datetime", "data_block_id"], suffix="_forecast")
              )
    
    data = (data
            .join(other=create_lagged_weather_forecast_features(weather_forecast), on=["county", "datetime", "data_block_id"], how="left")
            .join(other=create_lagged_historical_weather_week_features(historical_weather), on=["county", "datetime"], how="left")
            .join(other=create_lagged_historical_weather_day_features(historical_weather), on=["county", "datetime"], how="left")
            )

    n_day_lags = 7
    # Create revealed targets for all day lags
    revealed_targets = data.select("datetime", "client_id", "target")
    for day_lag in range(2, n_day_lags+1):
        data = data.join(revealed_targets.with_columns(pl.col("datetime") + pl.duration(days=day_lag)),
                          how="left",
                          on=["datetime", "client_id"],
                          suffix=f'_{day_lag}_days_ago'
                          )


    data = data.with_columns(
        pl.col("product_type").replace({0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}, default="Unknown"),
        pl.col("date").dt.strftime("%Y-%m-%d").is_in([x.strftime("%Y-%m-%d") for x  in ESTONIAN_HOLIDAYS]).alias("is_holiday")
    )

    data = data.with_columns(
        pl.col(pl.Int64).cast(pl.Int32),
        pl.col(pl.Float64).cast(pl.Float32),
    )

    return data.drop_nulls()

In [5]:
import pandas as pd
a = pd.DataFrame()
isinstance(a, pd.DataFrame)

True

In [ ]:
X: pd.DataFrame = create_training_data().collect().to_pandas()
X["noise"] = np.random.normal(0, 1, len(X)).astype(np.float32)

for col in X.columns:
    if X[col].dtype == "object":
        X[col] = X[col].astype("category")

In [ ]:
for col in sorted(X.columns):
    print(col, X[col].dtype)

## Train model function

In [ ]:
def feature_selection( data: pd.DataFrame) -> pd.DataFrame:
    return data.drop(columns=[
        "client_id",
        "data_block_id",
        "date",
        "date_client",
        "date_gas",
        "datetime",
        "county",
        "county_name_forecast",
        "latitude_min_forecast",
        "latitude_max_forecast",
        "longitude_min_forecast",
        "longitude_max_forecast",
        "hour",
        "quarter",
        "year"
    ]).copy(deep=True)

def train_model(dataframe: pd.DataFrame, train_indexes: list[int], test_indexes: list[int]) -> tuple:
    dataframe = feature_selection(dataframe)
    
    x_train, x_test = dataframe.loc[train_indexes], dataframe.loc[test_indexes]
    y_train, y_test = x_train.pop("target"), x_test.pop("target")

    eval_results = {}    
    model = lgb.LGBMRegressor(
        boosting_type='gbdt',
        num_leaves=31,
        max_depth=-1,
        learning_rate=0.1,
        n_estimators=1_000,
        subsample_for_bin=200_000,
        objective="huber",
        class_weight=None,
        min_split_gain=0.0,
        min_child_weight=0.001,
        min_child_samples=20,
        subsample=1.0,
        subsample_freq=0,
        colsample_bytree=1.0,
        reg_alpha=0.0,
        reg_lambda=0.0,
        random_state=SEED,
        n_jobs=-1,
        importance_type='split',
        linear_tree=True,
        verbosity=0,
        device="gpu",
        bagging_seed=SEED
    )

    model.fit(
        X=x_train, 
        y=y_train,
        eval_set=[(x_test, y_test)],
        eval_metric="mae",
        callbacks=[lgb.log_evaluation(), lgb.record_evaluation(eval_results), lgb.early_stopping(stopping_rounds=100)]
    )
    
    return model

## Training loop

In [ ]:
from typing import Generator

def train_test_split(dataframe: pd.DataFrame, datetime_col: str = "datetime", train_months: int = 3, test_months: int = 1) -> Generator:
    data_ = dataframe.copy(deep=True)
    data_["month_year"] = data_[datetime_col].dt.strftime('%Y-%m')
    unique_months = data_["month_year"].unique()
    unique_months.sort()
    for i, month_year in enumerate(unique_months[:-test_months], start=0):
        year, month = month_year.split("-")
        year, month = int(year), int(month)
        
        if i < train_months:
            train_month_start, train_month_end = unique_months[0], unique_months[i]
        else:
            train_month_start, train_month_end = unique_months[i-train_months+1], unique_months[i]
        test_month_start, test_month_end = unique_months[i + 1], unique_months[i + test_months]

        train = data_[(data_["month_year"] >= train_month_start) & (data_["month_year"] <= train_month_end)]
        train_index = pd.concat([train, data_.query("month == @month & year < @year")]).index
        
        test_index = data_[(data_["month_year"] >= test_month_start) & (data_["month_year"] <= test_month_end)].index
        
        yield train_index, test_index
        
        
def get_train_split_from_test(data, test_data, train_months: int = 3, test_months: int = 1):
    test_data_ = test_data.copy(deep=True)
    data_ = data.copy(deep=True)

    unique_month_year_test = test_data_[["year", "month"]].unique().sort_values(by=["year", "month"])
    
    for _, row in unique_month_year_test.iterrows():
        train = data_[(data_["year"] == row["year"]) & (data_["month"] >= row["month"] - train_months) & (data_["month"] < row["month"])]
        train_index = pd.concat([train, data_.query("month == @row.month & year < @row.year")]).index

        test_index = data_[(data_["year"] == row["year"]) & (data_["month"] == row["month"])]

        yield train_index, test_index


def training_loop(dataframe: pd.DataFrame):    
    models = {
        "consumer": [],
        "producer": []
    }
    for is_consumer in [0, 1]:
        client_type = "consumer" if is_consumer else "producer"
        print(f"Training for {client_type}")

        data = dataframe[dataframe.is_consumption == is_consumer].reset_index(drop=True).drop(columns=["is_consumption"]).copy(deep=True)

        for i, (train_index, test_index) in enumerate(train_test_split(data, datetime_col="datetime", train_months=15, test_months=6)):
            models[client_type].append(train_model(data, train_index, test_index))
    return models

models = training_loop(dataframe=X)

# Submission

In [ ]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

In [ ]:
# Reload enefit environment (only in debug mode, otherwise the submission will fail)
if DEBUG:
    enefit.make_env.__called__ = False
    type(env)._state = type(type(env)._state).__dict__['INIT']
    iter_test = env.iter_test()

In [ ]:
# List of target_revealed dataframes
previous_revealed_targets = []

for (test,
     revealed_targets,
     client_test,
     historical_weather_test,
     forecast_weather_test,
     electricity_test,
     gas_test,
     sample_prediction) in iter_test:

    # Rename test set to make consistent with train
    test = test.rename(columns = {'prediction_datetime': 'datetime'})

    # Initiate column data_block_id with default value to join on
    id_column = 'data_block_id'

    test[id_column] = 0
    gas_test[id_column] = 0
    electricity_test[id_column] = 0
    historical_weather_test[id_column] = 0
    forecast_weather_test[id_column] = 0
    client_test[id_column] = 0
    revealed_targets[id_column] = 0

    data_test = FeatureProcessor(
        data = test,
        client = client_test,
        historical_weather = historical_weather_test,
        forecast_weather = forecast_weather_test,
        electricity = electricity_test,
        gas = gas_test
    )

    # Store revealed_targets
    previous_revealed_targets.insert(0, revealed_targets)

    if len(previous_revealed_targets) == N_day_lags:
        previous_revealed_targets.pop()

    # Add previous revealed targets
    df_test = create_revealed_targets_test(data = data_test.copy(),
                                           previous_revealed_targets = previous_revealed_targets.copy(),
                                           N_day_lags = N_day_lags
                                           )

    # Make prediction
    X_test = df_test[features]
    sample_prediction['target'] = clf.predict(X_test)
    env.predict(sample_prediction)